
# Onboarding new cluster

This tutorial serves as a guide for adding cluster resources to Operate First repositories. Following steps in this guide should result in a PR against the operate-first/apps repository and operate-first/argocd-apps repository.


## Prerequisites

[x] - Cluster has to be imported to Operate First ACM, please raise an issue in the support repository [here](https://github.com/operate-first/support/issues).

## Recipe

If you want to know more about the overall design please consult the ADR documentation at [operate-first.cloud](https://www.operate-first.cloud/blueprints/blueprint/#architectural-decisions).
We store all the cluster-scoped resources in a `cluster-scope` folder in the `apps` repository. This folder is rendered via Kustomize and is consumed by ArgoCD based on application manifests located in `argocd-apps` repository.
This tutorial is split into two parts. Each part modifies a single repository and should result in a created pull request.

## Changing the apps repository

In this part of tutorial we prepare all files which are necessary for onboarding cluster in `apps` repository.

### Workspace setup

Please fork/clone the [operate-first/apps](https://github.com/operate-first/apps) repository. During this whole setup, we’ll be working within this repository.

1. Go to [operate-first/apps](https://github.com/operate-first/apps).
2. Click on a fork button.
3. When a fork is created click on the code button and copy an adress of your forked repository.
4. Run following command using copied adress:

In [14]:
!git clone https://github.com/$JUPYTERHUB_USER/apps.git
%cd apps

Cloning into 'apps'...
remote: Enumerating objects: 10668, done.
remote: Counting objects: 100% (349/349), done.
remote: Compressing objects: 100% (221/221), done.
remote: Total 10668 (delta 128), reused 321 (delta 114), pack-reused 10319
Receiving objects: 100% (10668/10668), 2.33 MiB | 924.00 KiB/s, done.
Resolving deltas: 100% (5239/5239), done.
/home/mdrla/work/operate-first/hitchhikers-guide/pages/apps



### Define important variables:


In [ ]:
UUID=!uuidgen
CLUSTER_NAME="my-cluster"
CLUSTER_KEYCLOAK_SECRET=UUID
CLUSTER_DESCRIPTION="Description of cluster"
CLUSTER_REGION="moc"
if CLUSTER_REGION == "moc":
    CLUSTER_LOCATION="na"
else:
    CLUSTER_LOCATION="emea"


### Define cluster in cluster-scope

For each cluster we have a separate overlay in the cluster-scope folder, grouped by region. For more information on this topic, see [ADR-0009 - Declarative Definitions for Cluster Scoped Resources](https://www.operate-first.cloud/blueprints/blueprint/docs/adr/0009-cluster-resources.md).\
To create base for cluster, start with creating files in cluster-scope which define groups and namespaces used in cluster.\
Create file `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/groups/cluster-admins.yaml` which will contain user names of users which will be considered admins of clusters. Please change the following manifest to contain the cluster admins username. Please use GitHub usernames in lowercase:

In [18]:
!mkdir -p cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/groups
input_text="""\
apiVersion: user.openshift.io/v1
kind: Group
metadata:
    name: cluster-admins
users:
    - user_admin_1
    - user_admin_2
"""
%store input_text  >cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/groups/cluster-admins.yaml 

Writing 'input_text' (str) to file 'cluster-scope/overlays/prod/moc/my-cluster/groups/cluster-admins.yaml'.


We use Kustomize to compose manifests. `kustomization.yaml` files serves as manifests for Kustomize instructing the tool which manifests to pull and how to overlay and render them together.
Create `kustomization.yaml` file in `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME`:

In [22]:
input_text="""\
---
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization

resources:
  - ../common
  - ../../../../base/user.openshift.io/groups/cluster-admins
patchesStrategicMerge:
  - groups/cluster-admins.yaml

"""
%store input_text  >cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/kustomization.yaml

Writing 'input_text' (str) to file 'cluster-scope/overlays/prod/moc/my-cluster/kustomization.yaml'.



### Enable ArgoCD management in ACM

Since the cluster is being managed by ACM, we leverage ACM's integration capabilities and use it to connect our ArgoCD through it. All we have to do is instruct ACM to enable the cluster to be managed by ArgoCD. This file is going to be located in `acm/overlays/$CLUSTER_REGION/infra/klusterletaddonconfigs`.


In [20]:
text_input="""\
apiVersion: agent.open-cluster-management.io/v1
kind: KlusterletAddonConfig
metadata:
  name: %s
  namespace: %s
spec:
  applicationManager:
    argocdCluster: true
    enabled: true
""" % (CLUSTER_NAME, CLUSTER_NAME)
%store text_input >acm/overlays/{CLUSTER_REGION}/infra/klusterletaddonconfigs/{CLUSTER_NAME}.yaml

Writing 'text_input' (str) to file 'acm/overlays/moc/infra/klusterletaddonconfigs/my-cluster.yaml'.


Next step is to list this manifest in `kustomization.yaml` file which is located in  `acm/overlays/$CLUSTER_REGION/infra/`.

In [21]:
!cd acm/overlays/{CLUSTER_REGION}/infra/ && kustomize edit  add resource klusterletaddonconfigs/{CLUSTER_NAME}.yaml


### Enable logging via SSO

The last thing that we have to setup is user access to the via Operate First SSO as the identity provider. This enables users to log in to the cluster console and API via Operate First SSO:\
Create a Keycloak client definition for your cluster in `keycloak/overlays/moc/infra/clients/$CLUSTER_NAME.yaml` and encrypt the file with sops. You can find the key to import from [here](https://github.com/operate-first/apps/tree/master/cluster-scope/overlays/prod/moc#secret-management):

The `KeycloakClient` resource makes our SSO aware of the cluster's presence - it configures and enables the cluster to a client to the SSO.
:::{note}
Don't forget to remove unecrypted file
:::


In [23]:
text_input="""\
apiVersion: keycloak.org/v3alpha1
kind: KeycloakClient
metadata:
    name: %s
    labels:
        client: %s
spec:
    client:
        clientId: %s
        defaultClientScopes:
            - profile
        description: %s
        name: %s cluster
        protocol: openid-connect
        secret: %s
        standardFlowEnabled: true
        redirectUris:
            - https://oauth-openshift.apps.%s.%s.operate-first.cloud/oauth2callback/operate-first
    realmSelector:
        matchLabels:
            realm: operate-first

""" %(CLUSTER_NAME, CLUSTER_NAME, CLUSTER_NAME, CLUSTER_DESCRIPTION, CLUSTER_NAME, CLUSTER_KEYCLOAK_SECRET, CLUSTER_NAME, CLUSTER_LOCATION)
%store text_input >keycloak/overlays/moc/infra/clients/{CLUSTER_NAME}.yaml
!sops --encrypt --encrypted-regex="^name|secret" --pgp="0508677DD04952D06A943D5B4DC4116D360E3276" keycloak/overlays/moc/infra/clients/{CLUSTER_NAME}.yaml >keycloak/overlays/moc/infra/clients/{CLUSTER_NAME}.enc.yaml
!rm keycloak/overlays/moc/infra/clients/{CLUSTER_NAME}.yaml

Writing 'text_input' (str) to file 'keycloak/overlays/moc/infra/clients/my-cluster.yaml'.
[PGP]	 WARN[0000] Deprecation Warning: GPG key fetching from a keyserver within sops will be removed in a future version of sops. See https://github.com/mozilla/sops/issues/727 for more information. 


Now we need to include this client resource in the `secret-generator` located at `keycloak/overlays/moc/infra/secret-generator.yaml` This file is a generator for Kustomize and decrypts the resources for us. We need to include the newly added client to the list of resources that should be decrypted and applied to the SSO configuration.


In [24]:
!yq e -i '.files += "clients/{CLUSTER_NAME}.enc.yaml"' keycloak/overlays/moc/infra/secret-generator.yaml


That concludes the configuration of the SSO server side. Now we need to configure the cluster we're onboarding to ask the SSO for user identities.

Create a `operate-first-sso-secret` secret resource which contains cluster's SSO credentials at `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/oauths/operate-first-sso-secret.yaml`

In [25]:
!mkdir -p cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/oauths/
text_input="""\
apiVersion: v1
kind: Secret
metadata:
    name: operate-first-sso-secret
    namespace: openshift-config
    annotations:
        argocd.argoproj.io/compare-options: IgnoreExtraneous
        argocd.argoproj.io/sync-options: Prune=false
type: Opaque
stringData:
    clientSecret: %s
""" % (CLUSTER_KEYCLOAK_SECRET)
%store text_input >cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/oauths/operate-first-sso-secret.yaml
!sops --encrypt --encrypted-regex="^(data|stringData)$" --pgp="0508677DD04952D06A943D5B4DC4116D360E3276" cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/oauths/operate-first-sso-secret.yaml >cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/oauths/operate-first-sso-secret.enc.yaml
!rm cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/oauths/operate-first-sso-secret.yaml

Writing 'text_input' (str) to file 'cluster-scope/overlays/prod/moc/my-cluster/oauths/operate-first-sso-secret.yaml'.
[PGP]	 WARN[0000] Deprecation Warning: GPG key fetching from a keyserver within sops will be removed in a future version of sops. See https://github.com/mozilla/sops/issues/727 for more information. 


Since this is an encrypted secret. We need to tell Kustomize how to access it and how to decrypt it. This means we need yet another `secret-generator.yaml`, similar to what we created above. Create `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/secret-generator.yaml`.

In [26]:
text_input="""\
---
apiVersion: viaduct.ai/v1
kind: ksops
metadata:
  name: secret-generator
files:
  - oauths/operate-first-sso-secret.enc.yaml
"""
%store text_input >cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/secret-generator.yaml

Writing 'text_input' (str) to file 'cluster-scope/overlays/prod/moc/my-cluster/secret-generator.yaml'.


And the last resouce we have to create for the SSO, is the `OAuth` configuration of OpenShift. This resource defines identity providers available to users when authenticating to the cluster.
Create `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/oauths/cluster_patch.yaml`:

In [27]:
text_input="""\
---
apiVersion: config.openshift.io/v1
kind: OAuth
metadata:
  name: cluster
spec:
  identityProviders:
    - mappingMethod: claim
      name: operate-first
      openID:
        claims:
          email:
            - email
          name:
            - name
          preferredUsername:
            - preferred_username
        clientID: %s
        clientSecret:
          name: operate-first-sso-secret
        extraScopes: []
        issuer: https://keycloak-keycloak.apps.moc-infra.massopen.cloud/auth/realms/operate-first
      type: OpenID
""" % (CLUSTER_NAME)
%store text_input >cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/oauths/cluster_patch.yaml

Writing 'text_input' (str) to file 'cluster-scope/overlays/prod/moc/my-cluster/oauths/cluster_patch.yaml'.


Yet again we need to tell Kustomize to import these additional resources to the new cluster. \
To do so, please list secret-generator, cluster patch and configuration of SSO in `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/kustomization.yaml`:

In [28]:
!cd cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME} && kustomize edit  add resource ../../../../base/config.openshift.io/oauths/cluster
!yq e -i -P '.generators = ["secret-generator.yaml"]' cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/kustomization.yaml
!yq e -i -P '.patchesStrategicMerge= ["oauths/cluster_patch.yaml"]' cluster-scope/overlays/prod/{CLUSTER_REGION}/{CLUSTER_NAME}/kustomization.yaml


### Finalize

Please stage your changes and send them as a PR against the [operate-first/apps](https://github.com/operate-first/apps) repository. 
:::{note}
Make sure that following files/ have been modified/added:
- [x] `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/groups/cluster-admins.yaml`
- [x] `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/kustomization.yaml`
- [x] `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/oauths/operate-first-sso-secret.enc.yaml`
- [x] `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/secret-generator.yaml`
- [x] `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME/oauths/cluster_patch.yaml`
- [x] `acm/overlays/moc/infra/klusterletaddonconfigs/$CLUSTER_NAME.yaml`
- [x] `acm/overlays/moc/infra/kustomization.yaml`
- [x] `keycloak/overlays/moc/infra/clients/$CLUSTER_NAME.enc.yaml`
- [x] `keycloak/overlays/moc/infra/secret-generator.yaml`
:::

In [41]:
!git status
!git add .
!git commit -m "feat(onboarding): Add cluster $CLUSTER_NAME"

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   acm/overlays/moc/infra/kustomization.yaml
	modified:   keycloak/overlays/moc/infra/secret-generator.yaml

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	acm/overlays/moc/infra/klusterletaddonconfigs/my-cluster.yaml
	cluster-scope/overlays/prod/moc/my-cluster/
	keycloak/overlays/moc/infra/clients/my-cluster.enc.yaml

no changes added to commit (use "git add" and/or "git commit -a")
[master 59ba074] feat(onboarding): Add cluster my-cluster
 9 files changed, 159 insertions(+)
 create mode 100644 acm/overlays/moc/infra/klusterletaddonconfigs/my-cluster.yaml
 create mode 100644 cluster-scope/overlays/prod/moc/my-cluster/groups/cluster-admins.yaml
 create mode 100644 cluster-scope/overlays/prod/moc/my-cluster/kustomiz


## Changing the argocd-apps repository

In previous section of this guide we enabled ArgoCD management and defined all the resources that should be applied by ArgoCD for us. Now we have tell ArgoCD about those resource. This is done via ArgoCD applications.
In Operate First we host all ArgoCD application resources in the `argocd-apps` repository.
Following applications will be defined in this section of the tutorial:
1. App-of-apps for deploying other ArgoCD applications to this cluster
2. Application which deploys the cluster management related manifests


### Workspace setup

Please fork/clone the [operate-first/argocd-apps](https://github.com/operate-first/argocd-apps) repository. During this whole setup, we’ll be working within this repository.

1. Go to [operate-first/argocd-apps](https://github.com/operate-first/argocd-apps).
2. Click on a fork button.
3. When a fork is created click on the code button and copy an adress of your forked repository.
4. Run following command using copied adress:

In [29]:
%cd ..
!git clone https://github.com/$JUPYTERHUB_USER/argocd-apps.git
%cd argocd-apps

/home/mdrla/work/operate-first/hitchhikers-guide/pages
Cloning into 'argocd-apps'...
remote: Enumerating objects: 1870, done.
remote: Counting objects: 100% (992/992), done.
remote: Compressing objects: 100% (524/524), done.
remote: Total 1870 (delta 603), reused 784 (delta 464), pack-reused 878
Receiving objects: 100% (1870/1870), 293.74 KiB | 1.35 MiB/s, done.
Resolving deltas: 100% (1203/1203), done.
/home/mdrla/work/operate-first/hitchhikers-guide/pages/argocd-apps



### Create the App-of-apps

First we will create the app-of-apps for this cluster. This application pattern allows us to simply define deploy other future ArgoCD applications that we expect to deploy to this cluster and have those application resources also managed through git-ops.

Create `app-of-apps/app-of-apps-$CLUSTER_NAME.yaml`

In [30]:
text_input="""\
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: opf-app-of-apps-%s
spec:
  destination:
    namespace: argocd
    name: moc-infra
  project: operate-first
  source:
    path: envs/%s/%s.yaml
    repoURL: https://github.com/operate-first/argocd-apps.git
    targetRevision: HEAD
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
    syncOptions:
    - Validate=false
    - ApplyOutOfSyncOnly=true
""" % (CLUSTER_NAME, CLUSTER_REGION, CLUSTER_NAME)
%store text_input >app-of-apps/app-of-apps-{CLUSTER_NAME}.yaml

Writing 'text_input' (str) to file 'app-of-apps/app-of-apps-my-cluster.yaml'.


List the file in `app-of-apps/kustomization.yaml`

In [31]:
!cd app-of-apps && kustomize edit  add resource app-of-apps-{CLUSTER_NAME}.yaml


### Cluster management application

As the next step we have to create an environment folder for ArgoCD application resource, where we define resources for this specific cluster. In this folder we can create a new application for cluster management. 
As you can see in the application manifest below in the `.spec.source`, the application points to the resources we created in the first section of this tutorial. The path is pointing ArgoCD to the `cluster-scope/overlays/prod/$CLUSTER_REGION/$CLUSTER_NAME` which is precisely where our cluster admin groups and OAuth changes are located.


In [33]:
!mkdir  -p envs/{CLUSTER_REGION}/{CLUSTER_NAME}/cluster-management
text_input="""\
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: cluster-resources
spec:
  destination:
    name: %s
    namespace: open-cluster-management-agent
  ignoreDifferences:
    - group: imageregistry.operator.openshift.io
      jsonPointers:
        - /spec/defaultRoute
        - /spec/httpSecret
        - /spec/proxy
        - /spec/requests
        - /spec/rolloutStrategy
      kind: Config
      name: cluster
  project: cluster-management
  source:
    path: cluster-scope/overlays/prod/%s/%s
    repoURL: https://github.com/operate-first/apps.git
    targetRevision: HEAD
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
    syncOptions:
    - Validate=false
    - ApplyOutOfSyncOnly=true

""" % (CLUSTER_NAME, CLUSTER_REGION, CLUSTER_NAME)
%store text_input >envs/{CLUSTER_REGION}/{CLUSTER_NAME}/cluster-management/cluster-resources.yaml

Writing 'text_input' (str) to file 'envs/moc/my-cluster/cluster-management/cluster-resources.yaml'.


Create `cluster-resources.yaml` in ` envs/$CLUSTER_REGION/$CLUSTER_NAME/cluster-management/`.

Since we use Kustomize for ArgoCD applications as well, we need to create a `kustomization.yaml` which lists `cluster-resources.yaml` here.

In [34]:
text_input="""\
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization
resources:
  - cluster-resources.yaml  
"""
%store text_input >envs/{CLUSTER_REGION}/{CLUSTER_NAME}/cluster-management/kustomization.yaml

Writing 'text_input' (str) to file 'envs/moc/my-cluster/cluster-management/kustomization.yaml'.


And additional `kustomization.yaml` which will list cluster-management resources. As you can see now we're creating this file at the `envs/$CLUSTER_REGION/$CLUSTER_NAME` path in the `argocd-apps` repository, which is exactly where the app of apps defined above points to.

In [35]:
text_input="""\
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization
namespace: argocd
nameSuffix: -%s
resources:
- cluster-management
""" % (CLUSTER_NAME)
%store text_input >envs/{CLUSTER_REGION}/{CLUSTER_NAME}/kustomization.yaml

Writing 'text_input' (str) to file 'envs/moc/my-cluster/kustomization.yaml'.



### Finalize

Please stage your changes and send them as a PR against the [operate-first/argocd-apps](https://github.com/operate-first/argocd-apps) repository. 
:::{note}
Make sure that following files/ have been modified/added:
- [x] `app-of-apps/app-of-apps-my-$CLUSTER_NAME.yaml`
- [x] `app-of-apps/kustomization.yaml`
- [x] `envs/$CLUSTER_REGION/$CLUSTER_NAME/cluster-management/cluster-resources.yaml`
- [x] `envs/$CLUSTER_REGION/$CLUSTER_NAME/cluster-management/kustomization.yaml`
- [x] `envs/$CLUSTER_REGION/$CLUSTER_NAME/kustomization.yaml`
:::

In [51]:
!git status
!git add .
!git commit -m "feat(onboarding): Add cluster $CLUSTER_NAME"
!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   app-of-apps/kustomization.yaml

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	app-of-apps/app-of-apps-my-cluster.yaml
	envs/moc/my-cluster/

no changes added to commit (use "git add" and/or "git commit -a")
[main e9d798f] feat(onboarding): Add cluster my-cluster
 5 files changed, 61 insertions(+)
 create mode 100644 app-of-apps/app-of-apps-my-cluster.yaml
 create mode 100644 envs/moc/my-cluster/cluster-management/cluster-resources.yaml
 create mode 100644 envs/moc/my-cluster/cluster-management/kustomization.yaml
 create mode 100644 envs/moc/my-cluster/kustomization.yaml
